In [1]:
import pandas as pd

csvpath = "Resources/sfpd_incidents_2016.csv"
sfpd_df = pd.read_csv(csvpath)

In [2]:
# Cleaned 'Other Offenses' data frame  
sf_other_off = sfpd_df[~sfpd_df['Category'].isin(['NON-CRIMINAL','SECONDARY CODES','WARRANTS','SUICIDE','BRIBERY','EXTORTION','EXTORTION','FAMILY OFFENSES','LOITERING','SEX OFFENSES', 'NON FORCIBLE','BAD CHECKS','GAMBLING','PORNOGRAPHY/OBSCENE MAT','TREA'])]
other_off_clean = sf_other_off.dropna(how='any')
other_off_drop = other_off_clean.drop(['PdId'], axis = 1)
other_off = other_off_drop.loc[other_off_drop["Category"]== "OTHER OFFENSES"]
other_off

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location
6,160003130,OTHER OFFENSES,PAROLE VIOLATION,Saturday,01/02/2016 12:00:00 AM,00:04,SOUTHERN,"ARREST, BOOKED",MARY ST / HOWARD ST,-122.405721,37.780879,"(37.7808789360214, -122.405721454567)"
12,160073014,OTHER OFFENSES,RESISTING ARREST,Monday,01/25/2016 12:00:00 AM,23:20,BAYVIEW,"ARREST, BOOKED",200 Block of HUNTERSPOINT BL,-122.376758,37.735697,"(37.7356970275482, -122.37675765553)"
23,160005841,OTHER OFFENSES,PROBATION VIOLATION,Sunday,01/03/2016 12:00:00 AM,06:50,MISSION,"ARREST, BOOKED",23RD ST / FOLSOM ST,-122.414234,37.754099,"(37.7540986882068, -122.414233849038)"
25,160006598,OTHER OFFENSES,VIOLATION OF STAY AWAY ORDER,Sunday,01/03/2016 12:00:00 AM,15:09,TARAVAL,"ARREST, BOOKED",0 Block of CAMBON DR,-122.474445,37.718302,"(37.718302204766, -122.474444639595)"
26,160006770,OTHER OFFENSES,LOST/STOLEN LICENSE PLATE,Sunday,01/03/2016 12:00:00 AM,10:30,MISSION,NONE,16TH ST / LANDERS ST,-122.427563,37.764575,"(37.7645752317615, -122.427562596985)"
...,...,...,...,...,...,...,...,...,...,...,...,...
150392,160659761,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,08/15/2016 12:00:00 AM,19:06,BAYVIEW,"ARREST, BOOKED",16TH ST / 3RD ST,-122.389103,37.766904,"(37.7669037251921, -122.389103032813)"
150399,160659846,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Monday,08/15/2016 12:00:00 AM,20:15,PARK,"ARREST, BOOKED",7TH AV / HUGO ST,-122.464237,37.765066,"(37.7650658096524, -122.464236532783)"
150422,160784625,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Tuesday,09/27/2016 12:00:00 AM,12:55,MISSION,"ARREST, BOOKED",TREAT AV / ALAMEDA ST,-122.412424,37.768074,"(37.7680741693848, -122.412424364002)"
150438,160874408,OTHER OFFENSES,TRAFFIC VIOLATION,Thursday,10/27/2016 12:00:00 AM,04:15,PARK,NONE,DUBOCE AV / NOE ST,-122.433575,37.769177,"(37.7691767476277, -122.433575097282)"


In [3]:
# The data frame with all unwanted categories removed
sf_clean = sfpd_df[~sfpd_df['Category'].isin(['NON-CRIMINAL', 'OTHER OFFENSES','SECONDARY CODES','WARRANTS','SUICIDE','BRIBERY','EXTORTION','EXTORTION','FAMILY OFFENSES','LOITERING','SEX OFFENSES', 'NON FORCIBLE','BAD CHECKS','GAMBLING','PORNOGRAPHY/OBSCENE MAT','TREA'])]
sf_clean_df = sf_clean.dropna(how='any')
sf_crime = sf_clean_df.drop(['PdId'], axis = 1)
sf_crime

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)"
5,160002869,ASSAULT,BATTERY,Friday,01/01/2016 12:00:00 AM,21:35,NORTHERN,NONE,1700 Block of BUSH ST,-122.426077,37.788019,"(37.788018555829, -122.426077177375)"
9,160003641,MISSING PERSON,FOUND PERSON,Friday,01/01/2016 12:00:00 AM,10:06,BAYVIEW,NONE,100 Block of CAMERON WY,-122.387182,37.720967,"(37.7209669615499, -122.387181635995)"
10,160086863,LARCENY/THEFT,ATTEMPTED THEFT FROM LOCKED VEHICLE,Friday,01/29/2016 12:00:00 AM,22:30,TARAVAL,NONE,1200 Block of 19TH AV,-122.477377,37.764478,"(37.7644781578695, -122.477376524003)"
...,...,...,...,...,...,...,...,...,...,...,...,...
150492,160823994,ASSAULT,BATTERY,Sunday,10/09/2016 12:00:00 AM,21:15,MISSION,NONE,18TH ST / MISSION ST,-122.419360,37.761836,"(37.7618358012376, -122.419360352761)"
150493,160858327,ASSAULT,BATTERY,Friday,10/21/2016 12:00:00 AM,13:30,MISSION,NONE,3400 Block of 18TH ST,-122.419749,37.761867,"(37.7618672648972, -122.419748993977)"
150495,161061000,ASSAULT,BATTERY,Friday,12/30/2016 12:00:00 AM,21:01,PARK,NONE,OAK ST / STANYAN ST,-122.453982,37.771428,"(37.7714278595913, -122.453981622365)"
150497,176000758,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Thursday,12/29/2016 12:00:00 AM,20:00,CENTRAL,NONE,PINE ST / TAYLOR ST,-122.412269,37.790673,"(37.7906727649886, -122.41226909106)"


In [4]:
# Cleaning the 'Time' column
time_min = pd.to_datetime(sf_crime["Time"])
time_clean = time_min.dt.hour
sf_crime['Hour'] = time_clean
sf_crime

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,Hour
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",11
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",11
5,160002869,ASSAULT,BATTERY,Friday,01/01/2016 12:00:00 AM,21:35,NORTHERN,NONE,1700 Block of BUSH ST,-122.426077,37.788019,"(37.788018555829, -122.426077177375)",21
9,160003641,MISSING PERSON,FOUND PERSON,Friday,01/01/2016 12:00:00 AM,10:06,BAYVIEW,NONE,100 Block of CAMERON WY,-122.387182,37.720967,"(37.7209669615499, -122.387181635995)",10
10,160086863,LARCENY/THEFT,ATTEMPTED THEFT FROM LOCKED VEHICLE,Friday,01/29/2016 12:00:00 AM,22:30,TARAVAL,NONE,1200 Block of 19TH AV,-122.477377,37.764478,"(37.7644781578695, -122.477376524003)",22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150492,160823994,ASSAULT,BATTERY,Sunday,10/09/2016 12:00:00 AM,21:15,MISSION,NONE,18TH ST / MISSION ST,-122.419360,37.761836,"(37.7618358012376, -122.419360352761)",21
150493,160858327,ASSAULT,BATTERY,Friday,10/21/2016 12:00:00 AM,13:30,MISSION,NONE,3400 Block of 18TH ST,-122.419749,37.761867,"(37.7618672648972, -122.419748993977)",13
150495,161061000,ASSAULT,BATTERY,Friday,12/30/2016 12:00:00 AM,21:01,PARK,NONE,OAK ST / STANYAN ST,-122.453982,37.771428,"(37.7714278595913, -122.453981622365)",21
150497,176000758,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Thursday,12/29/2016 12:00:00 AM,20:00,CENTRAL,NONE,PINE ST / TAYLOR ST,-122.412269,37.790673,"(37.7906727649886, -122.41226909106)",20


In [5]:
# Bin code 00:00-08:00, 09:00-17:00, 18:00-24:00

In [6]:
# Creating the bins for the 'Hours'
bins = [0,8,17,24]
time_of_day = ["Morning", "Middle", "Night"]

In [7]:
# Applying the bins to 'Hours'
pd.cut(sf_crime["Hour"], bins, labels=time_of_day).head()

0     Middle
1     Middle
5      Night
9     Middle
10     Night
Name: Hour, dtype: category
Categories (3, object): [Morning < Middle < Night]

In [8]:
# Adding the assigning the bins to 'Time of Day' and adding it to the dataframe
sf_crime["Time of Day"] = pd.cut(sf_crime["Hour"], bins, labels=time_of_day)
sf_crime.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,Hour,Time of Day
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",11,Middle
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",11,Middle
5,160002869,ASSAULT,BATTERY,Friday,01/01/2016 12:00:00 AM,21:35,NORTHERN,NONE,1700 Block of BUSH ST,-122.426077,37.788019,"(37.788018555829, -122.426077177375)",21,Night
9,160003641,MISSING PERSON,FOUND PERSON,Friday,01/01/2016 12:00:00 AM,10:06,BAYVIEW,NONE,100 Block of CAMERON WY,-122.387182,37.720967,"(37.7209669615499, -122.387181635995)",10,Middle
10,160086863,LARCENY/THEFT,ATTEMPTED THEFT FROM LOCKED VEHICLE,Friday,01/29/2016 12:00:00 AM,22:30,TARAVAL,NONE,1200 Block of 19TH AV,-122.477377,37.764478,"(37.7644781578695, -122.477376524003)",22,Night


In [ ]:
# Bar graph showing crime types across the city 

In [ ]:
# Quartly breakdown of crimes (Michelle)